# Java Flight Recorder vs. gc log analysis
### Ellis Brown  (6/7/2021)

#### This notebook hopes to highlight the timing discrepencies between Zulu Mission Control's java flight recorder, and the output times in the log.

> important files: 
> - zulu_output_process: parses a CSV file using a regex string to extract gc pause times from a jfr file inspected using zulu mission control
> - process_log handles parsing and analysis of a log file




In [4]:
# In this cell, I hope to be able to analyze the difference between this data and Java/Zulu Flight Recorder
from scripts import zulu_output_process as zul
from scripts import process_log as pl
import math 
pl.setLogPath("datasets/long_amzn_workload.log") # set the file to analyze
pl.setLogSchema(0)                               # set the log schema (default 0)
# helper function to remove the last character
def remove_last_char(item):
    return float(item[:-1])
######
# set to true or false
print_differences = False



# Access the pause information
pause_table = pl.getYoungPauses(create_csv = False)

# Extract the column that has the pause duration
mine = pause_table[-2] # TODO: remove dependency on index information

# Access the pause information from flight recorder
with open("datasets/zulu_pauses_jfr.csv") as reader:
    zulu = reader.readlines()

# remove the '\n' character from each line
zulu = list(map(remove_last_char, zulu))

# calculate differences
## IMPORTANT NOTE: Zulu flight recorder considers concurrent pause/recycle ONE pause, mine TWO.
## Handle this case before using data to make conclusions
# possible solution 1: 
# If we see in pause_table[-1] the pattern 1 then 2, or 2 then 1, combine those values in 'mine'
# solution 2: Temporarily modify the file "process_log.py" to stop collecting one of the two pauses
table = []
sum_zulu = 0
sum_mine = 0
for idx in range(min(len(zulu),len(mine))):
    difference = float(zulu[idx]) - float(mine[idx])
    if (difference > 1 and print_differences):
        print("Difference: ", difference, end="")
        print("\tIndex : ", idx)
    table.append([mine[idx], zulu[idx], difference ])
    sum_zulu += zulu[idx]
    sum_mine += mine[idx]

ma = list(([val[2] for val in table]))
# Print collected data
print("Sum zulu: \t", round(sum_zulu, 4), "\n")
print("Sum mine: \t", round(sum_mine, 4), "\n")
print("Difference : \t", round(sum_zulu - sum_mine, 4), "\n")
print("(Difference / zulu) * 100: \t", round(100 * ((sum_zulu - sum_mine) / sum_zulu), 4), "\n")
print("Number of pauses", len(zulu),"\n")
print("Max difference in recorded times: ", round(max(ma), 4), "\n")
print("Average time difference (absolute value)", round(math.sqrt(sum([val * val for val in ma])) / 128, 4))

# print off a table of results per line
if print_differences:
    print("\n\n| Mine       | Zulu | Difference (ms)")

    for line in table:
        print(str(line[0]) +" ", "\t", line[1], "\t", line[2])




Sum zulu: 	 31463.336 

Sum mine: 	 31142.991 

Difference : 	 320.345 

(Difference / zulu) * 100: 	 1.0182 

Number of pauses 330 

Max difference in recorded times:  79.749 

Average time difference (absolute value) 0.9271


| Mine       | Zulu | Difference (ms)


NameError: name 't' is not defined